In [ ]:
from clean_data import dataset_ball, dataset_detections
from pathlib import Path

In [ ]:
ball_url = Path("C:/Users/mijin/OneDrive/Desktop/data_science/project/datasets/volleyball_ball_annotation")
balldata = dataset_ball(ball_url)
print(balldata.head())

   video_id  frame_id  frame_time  x_coord  y_coord  ball_visible
0         0     13286           0     1006      229          True
1         0     13336           0      955      148          True
2         0     13361           0      724      366          True
3         0     13406           0      471      160          True
4         0     13456           0      527       70          True


In [ ]:
actions_url = Path("C:/Users/mijin/OneDrive/Desktop/data_science/project/datasets/volleyball-detections")
actionsdata = dataset_detections(actions_url)
print(actionsdata.head())